In [4]:
import sys
sys.path.insert(0, '..')

import torch.nn as nn
from torch.utils.data import DataLoader
from utils.eye_dataset import *
from eye_classifier import *
import torchvision.transforms as transforms
import torchvision.models as models

### The Training Dataset

We use the utility class EyeImageDataset() to load the training dataset based on the metadata CSV file and
the target image folder

The training images will be loaded as needed due to memory constraints.

Since we're using the ALEXNET network, we need to resize our images to 224x224, so we apply some transforms on our dataset.

We also need to proper normalize the dataset accordingly to the ALEXNET input expectations.

In [5]:
base_dir = "../../data"

image_dir_training = f"{base_dir}/ODIR-5K/training"
#image_dir_training = f"{base_dir}/preprocessed_images"
image_dir_testing = f"{base_dir}/ODIR-5K/testing"
csv_file = f'{base_dir}/ODIR-5K/data.csv'

print ('reading input dataset')
input_size = 224

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_training, data_info_csv_file=csv_file, transform=apply_transforms)

reading input dataset


### Building the model

To build the model, we use the utility superclass EyeClassifier. 

We create a subclass named ResnetEyeClassifier and then we feed it with the intended model on its __init__ constructor.

Here we're building a resnet18 as the first layer (in fact resnet18 is comprised of many layers), followed by three linear fully connected layers
for image classification

The last layer will not have a transfer function because we opted to use CrossEntropyLoss as the loss function. 

You can change the loss function by using set_loss_function() if you wish to


In [6]:

class ResnetEyeClassifier(EyeClassifier):
    def __init__(self, num_classes: int) -> None:
        super(ResnetEyeClassifier, self).__init__(model=[

            (models.AlexNet(num_classes=num_classes), TransferFunction.NotApplicable),
        ])

nn = ResnetEyeClassifier(num_classes=len(ds.classes))
print(nn)

ResnetEyeClassifier(
  (layer 1): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=Fa

### Training the model

Training the model is very easy. All we have to do is to call train_model() passing the EyeImageDataset object. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [7]:
#ds.set_buffer_size(1024)
nn.train_model(ds, shuffle=True, gpu=False)


training (1%) epoch 1/100, loss = 2.0901


KeyboardInterrupt: 

### Saving some weights

You can use the util methods save/load weights to save or load individual model layer weights or the whole model weights,
so you can try different kinds of training.

You can also freeze/unfreeze layers, prevent them from being trained, by using the freeze_layer() unfreeze_layer() methods.

In [ ]:

#nn.save_layer_weights(0, "eye_classification_net_full_resnet18.w")
#nn.save_weights("eye_classification_net_full.w")


### Testing the model

Testing the model is similar to training it. All we have to do is to call test_model() passing a EyeImageDataset object pointing to the test dataset images. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [ ]:
input_size = 224

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_testing, data_info_csv_file=csv_file, transform=apply_transforms)


In [ ]:
ds.set_buffer_size(16)
nn.test_model(ds, gpu=False)

testing 1% [4 / 6564 files]
testing 2% [72 / 6564 files]
testing 3% [140 / 6564 files]
testing 4% [208 / 6564 files]
testing 5% [276 / 6564 files]
testing 6% [344 / 6564 files]
testing 7% [412 / 6564 files]
testing 8% [480 / 6564 files]
testing 9% [548 / 6564 files]
testing 10% [616 / 6564 files]
testing 11% [684 / 6564 files]
testing 12% [752 / 6564 files]
testing 13% [820 / 6564 files]
testing 14% [888 / 6564 files]
testing 15% [956 / 6564 files]
testing 16% [1024 / 6564 files]
testing 17% [1092 / 6564 files]
testing 18% [1160 / 6564 files]
testing 19% [1228 / 6564 files]
testing 20% [1296 / 6564 files]
testing 21% [1364 / 6564 files]
testing 22% [1432 / 6564 files]
testing 23% [1500 / 6564 files]
testing 24% [1568 / 6564 files]
testing 25% [1636 / 6564 files]
testing 26% [1704 / 6564 files]
testing 27% [1772 / 6564 files]
testing 28% [1840 / 6564 files]
testing 29% [1908 / 6564 files]
testing 30% [1976 / 6564 files]
testing 31% [2044 / 6564 files]
testing 32% [2112 / 6564 files]
tes